In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
from __future__ import print_function
import string
import json 
import jieba
import nltk
from opencc import OpenCC
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  
epochs = 20  
latent_dim = 256  
num_samples = 500


def readdata():
    cc = OpenCC('s2t')
    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()
    with open('/content/drive/MyDrive/translation2019zh_train.json', 'r', encoding='utf-8-sig') as f:
        words = f.readlines()
        for i, text in enumerate(words):
          if i>num_samples:
            break
          js = json.loads(text)
          js["chinese"] = cc.convert(js["chinese"])
          input_text = js["english"]
          target_text = js["chinese"]
          input_text = nltk.word_tokenize(text=input_text,language="english")
          target_text = jieba.cut(target_text.translate(str.maketrans('', '', string.punctuation)))
          target_text = [word for word in target_text if word != ' ']
          input_texts.append(input_text)
          target_texts.append(target_text)
          for char in input_text:
              input_characters.add(char)
          for char in target_text:
              target_characters.add(char)
    return input_texts,target_texts,input_characters,target_characters

input_texts,target_texts,input_characters,target_characters = readdata()
          
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 501
Number of unique input tokens: 3988
Number of unique output tokens: 4476
Max sequence length for inputs: 54
Max sequence length for outputs: 61


In [74]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
print(input_token_index['trade'])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
print(target_token_index['交易'])

681
1029


In [75]:


encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [76]:

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
          
          

model.save('s2s.h5')


encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
    
 
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

Epoch 1/20
7/7 [==============================] - 7s 414ms/step - loss: 2.7379 - val_loss: 2.5274
Epoch 2/20
7/7 [==============================] - 2s 239ms/step - loss: 2.4414 - val_loss: 2.5614
Epoch 3/20
7/7 [==============================] - 2s 241ms/step - loss: 2.4201 - val_loss: 2.5807
Epoch 4/20
7/7 [==============================] - 2s 242ms/step - loss: 2.4110 - val_loss: 2.6007
Epoch 5/20
7/7 [==============================] - 2s 244ms/step - loss: 2.4058 - val_loss: 2.6256
Epoch 6/20
7/7 [==============================] - 2s 246ms/step - loss: 2.4038 - val_loss: 2.6441
Epoch 7/20
7/7 [==============================] - 2s 238ms/step - loss: 2.3992 - val_loss: 2.6535
Epoch 8/20
7/7 [==============================] - 2s 242ms/step - loss: 2.4066 - val_loss: 2.6468
Epoch 9/20
7/7 [==============================] - 2s 243ms/step - loss: 2.3874 - val_loss: 2.6817
Epoch 10/20
7/7 [==============================] - 2s 241ms/step - loss: 2.3863 - val_loss: 2.6887
Epoch 11/20
7/7 [==

In [77]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
print(reverse_input_char_index)
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())
print(reverse_target_char_index)

{0: 'Christina', 1: 'Determining', 2: 'Aquino', 3: 'intuitively', 4: 'adaptable', 5: 'ldquo', 6: 'cleaning', 7: 'baleen', 8: 'MDA', 9: 'school', 10: 'come', 11: 'Virginia', 12: 'effect', 13: 'respectively', 14: 'LVDT', 15: 'Seonyudo', 16: 'provincial', 17: 'requirement', 18: 'specified', 19: 'Body', 20: 'doses', 21: 'Side-to-Side', 22: 'light', 23: 'linearly', 24: 'play', 25: 'revitalizing', 26: 'Farmers', 27: 'tests', 28: 'curve-fitting', 29: 'stages', 30: 'selectivity', 31: 'early', 32: 'pilings', 33: 'negotiation', 34: 'Seasoning', 35: 'Korean', 36: 'surprises', 37: 'scientists', 38: 'hypocrisy', 39: 'bedroom', 40: 'take', 41: 'positive', 42: 'making', 43: 'Basin', 44: 'welfare', 45: 'lemons', 46: 'articles', 47: 'lighting', 48: '玛雅人认为他们祖先基本上是从玉米进化而来的，由黄玉米与白玉米面团做成的。', 49: 'module', 50: 'infants', 51: 'inserted', 52: 'suggested', 53: 'backfill', 54: 'space', 55: 'proposals', 56: 'guacamole', 57: 'enjoyable', 58: 'retirees', 59: "'ve", 60: 'may', 61: 'Janice', 62: 'Isay', 63: 'count',

In [78]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


*
Input sentence: ['For', 'greater', 'sharpness', ',', 'but', 'with', 'a', 'slight', 'increase', 'in', 'graininess', ',', 'you', 'can', 'use', 'a', '1:1', 'dilution', 'of', 'this', 'developer', '.']
Decoded sentence: 的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: ['He', 'calls', 'the', 'Green', 'Book', ',', 'his', 'book', 'of', 'teachings', ',', '“', 'the', 'new', 'gospel', '.']
Decoded sentence: 的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: ['And', 'the', 'light', 'breeze', 'moves', 'me', 'to', 'caress', 'her', 'long', 'ear']
Decoded sentence: 的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: ['They', 'have', 'the', 'blood', 'of', 'martyrs', 'is', 'the', 'White', 'to', 'flow', '…']
Decoded sentence: 的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
*
Input sentence: ['Finally', ',', 'the', 'Lakers', 'head', 'to', 'the', 'Motor', 'City', 'to', 'take', 'on', 'a', 'Pistons', '